In [1]:
import ray
ray.__version__

/home/info/miniconda3/envs/dev10/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-30 02:00:34,934	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


'2.35.0'

In [10]:

!python --version

Python 3.12.4


In [5]:
from lib.db_io import Engine

import pandas as pd
import numpy as np
import glob
import warnings
warnings.filterwarnings(action='ignore')

db_info = {'host': '34.123.100.177',
          'port': '5432',
          'db_name': 'postgres',
          'user_name': 'tgsociety',
          'password': 'tgsociety'}


engine = Engine(db_info)

data = engine.select(table = 'history_학습id', schema='datamart', columns = ['train_id']
                     # , wheres = wheres
                     , orderbys = ['"등록일시" desc'])
train_id = data.loc[0,'train_id'].item()
path = glob.glob('/home/info/airflow-docker/dags/model_train_deploy/model/{}/*'.format(str(train_id)))[0]
path

'/home/info/airflow-docker/dags/model_train_deploy/model/4/best_model.torch'

In [6]:
train_id

4

In [7]:
from lib.sac_lstm import SACAgent
from lib.simulator_03 import Simulator_01
import time

# Hyperparameters
action_dim = 25
hidden_dim = 256
lr = 3e-4
batch_size = 64
num_episodes = 1000
device = 'cpu'


simulation_start_dt = '2025-02-01 07:00:00+09'
simulation_end_dt = '2025-03-31 23:00:00+09'

data_start_date = '2022-09-01 07:00:00+09'
data_end_date = '2024-09-01 23:00:00+09'

simulators = []
모델응답속도s = []
for try_num in range(3):
    모델응답속도s.append([])
    simulators.append(Simulator_01(
                                 simulation_start_dt = simulation_start_dt, 
                                 simulation_end_dt = simulation_end_dt,
        data_start_date = data_start_date,
                                 data_end_date = data_end_date,
                                 db_info = db_info,
                                 is_first = True,
                                시군구_ids = ['11220', '11230', '11240'], 차량숫자 = 15, try_num = try_num, mode = 'offline_optimal'))
    
    
    
    agent = SACAgent(action_dim = action_dim, hidden_dim = hidden_dim, lr = lr,  device = device, num_cars = len(simulators[try_num].차량데이터))
    agent.load_model(path = path)
    
    state = simulators[try_num].reset(선택최적 = 0, 선택rule = 'ruleX', mode = 'offline_optimal')
    done = False

    day_경과 = 0
    while simulators[try_num].simulation_done == False:
        st = time.time()
        actions = agent.select_action(state)
        모델응답속도s[try_num].append(time.time() - st)
        next_state, reward, done = simulators[try_num].step(actions)
        
        if (done[0] == 1):
            day_경과 += 1
    
        state = next_state



RuntimeError: Error(s) in loading state_dict for Actor:
	size mismatch for flatten_variable.weight: copying a param with shape torch.Size([256, 375]) from checkpoint, the shape in current model is torch.Size([256, 525]).

In [ ]:
최적대비성능비율s = []
for i in range(3):
    print('시행 {}'.format(i))
    수업수_총합 = np.sum(list(simulators[i].일자별_수업수.values()))
    차량대수_총합 = np.sum(list(simulators[i].일자별_가동차량수.values()))
    최적_수업수_총합 = np.sum(list(simulators[i].일자별_최적_수업수.values()))
    최적_차량대수_총합 = np.sum(list(simulators[i].일자별_최적_가동차량수.values()))
    
    print('강화학습 성능: {}'.format(수업수_총합 / 차량대수_총합))
    print('사후최적 성능: {}'.format(최적_수업수_총합 / 최적_차량대수_총합))
    최적대비성능비율s.append((수업수_총합 / 차량대수_총합) / (최적_수업수_총합 / 최적_차량대수_총합))
    print('최적대비성능비율: {}'.format(최적대비성능비율s[-1]))
print('평균: {}'.format(np.mean(최적대비성능비율s)))

In [ ]:
차량배정모델성능s = []
for i in range(3):
    print('시행 {}'.format(i))
    차량배정모델성능s.append(np.sum(list(simulators[i].일자별_차량당_이동거리.values())) / len(simulators[i].일자별_차량당_이동거리))
    print('차량배정모델성능: {}'.format(차량배정모델성능s[-1]))
print('평균: {}'.format(np.mean(차량배정모델성능s)))

In [ ]:
tmp = []
for i in range(3):
    print('시행 {}'.format(i))
    print('차량배정모델성능: {}'.format(np.mean(simulators[i].모델_응답속도)))
    tmp += simulators[i].모델_응답속도
print('평균: {}'.format(np.mean(tmp)))

In [ ]:
simulator.mode

In [ ]:
asdg

In [ ]:
state = simulator.reset(선택최적 = 0, 선택rule = 'ruleX')
done = False
episode_reward = 0
day_경과 = 0
while simulator.simulation_done == False:
    actions = agent.select_action(state)
    next_state, reward, done = simulator.step(actions)
    
    if (done[0] == 1):
        day_경과 += 1

    state = next_state
    episode_reward += reward[0]

In [ ]:
self.일자별_최적_수업수 = {}
self.일자별_최적_가동차량수 = {}
self.일자별_수업수 = {}
self.일자별_가동차량수 = {}

In [ ]:

import pandas as pd
import numpy as np
import os
import sys
from scipy.stats import ks_2samp
from lib.utils import 구분데이터붙이기, 군집나누기


st_dt = '2022-10-01 07:00:00+09'
ed_dt = '2024-08-21 23:00:00+09'

for try_num in range(3):
    simulator = Simulator_01(
                                 simulation_start_dt = st_dt, 
                                 simulation_end_dt = ed_dt,
        data_start_date = st_dt,
                                 data_end_date = ed_dt,
                                 db_info = db_info,
                                 is_first = True,
                                시군구_ids = ['11220', '11230', '11240'], 차량숫자 = 15, try_num = try_num)


In [ ]:
ks_2samp(ref_counts_list, cur_counts_list)[1].item()

In [ ]:
p_values

In [ ]:
len(list(군집_cur_data))

In [ ]:
군집_ref_data